# Serving Runtime Remote Artifacts

In [ ]:
import mlrun
import os

# Initialize the MLRun project object
project = mlrun.get_or_create_project('remote-artifacts',user_project=True,context='./')

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS, S3_BUCKET
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID', None)
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY', None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None)

# Setting as env variables
os.environ['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None, "please provide credentials"

secrets = {'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
           'AWS_SECRET_ACCESS_KEY':AWS_SECRET_ACCESS_KEY,
           'GOOGLE_APPLICATION_CREDENTIALS': GOOGLE_APPLICATION_CREDENTIALS}

project.set_secrets(secrets=secrets, provider='kubernetes')

S3_BUCKET = os.environ.get('S3_BUCKET', 'testbucket-igz-temp')
GS_BUCKET = os.environ.get('GS_BUCKET', 'testbucket-igz')

assert project.artifact_path != None, 'please run 01_mlrun_job.ipynb'

In [ ]:
# Setting dummy serving function
project.set_function(mlrun.new_function(name='serving', kind='serving',image='mlrun/mlrun', command = []))
project.get_function('serving').spec.build.commands = ['pip install xgboost']
project.get_function('serving').set_envs(secrets)
# adding our remote source model
model_artifact = project.get_artifact('log-transactions-log-transactions_clf_model')
model_path = model_artifact.get_target_path() + model_artifact.model_file

project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)

# mounting
project.get_function('serving').apply(mlrun.auto_mount())
project.get_function('serving').apply(mlrun.platforms.mount_s3())

In [ ]:
server = project.get_function('serving').to_mock_server()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

server.test(body={'inputs': X_test.tolist()[:3]})

In [ ]:
project.deploy_function('serving')

In [ ]:
import time
time.sleep(5)
project.get_function('serving').invoke('/v2/models/my_model/infer', body={'inputs': X_test.tolist()[:3]})

# Pipeline remote access

In [ ]:
%%writefile workflow.py
import mlrun
from kfp import dsl
import os

@dsl.pipeline(name="Mask Detection Pipeline")

def kfpipeline():
    # Get our project object:
    project = mlrun.get_current_project()
    
    project.get_function('log_transactions').apply(mlrun.platforms.set_env_variables({'GOOGLE_APPLICATION_CREDENTIALS':
                                                                                      os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')}))
    project.get_function('log_transactions').apply(mlrun.platforms.mount_s3())
    project.get_function('log_transactions').apply(mlrun.auto_mount())
    run = project.run_function('log_transactions')
    
    model_artifact = project.get_artifact('log-transactions-log-transactions_clf_model')
    model_path = model_artifact.get_target_path() + model_artifact.model_file
    
    project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)
    
    project.deploy_function('serving').after(run)
    return

In [ ]:
# Register the workflow file:
workflow_name = "iris_workflow"
project.set_workflow(workflow_name, "workflow.py")

# Save the project:
project.save()

In [ ]:
# Running with S3
project.run(name=workflow_name,watch=True, local=True, artifact_path=project.artifact_path)
project.save()

In [ ]:
# Running with GCS
project.run(name=workflow_name,watch=True, local=True, artifact_path='gs://' + GS_BUCKET)